Import Libraries

In [ ]:
import os
import re
import time
import spacy
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torchtext import data
import torch.optim as optim
from torchtext import vocab
import torch.nn.functional as F
from torchtext.data import Field
from torch.autograd import Variable
from torchtext.vocab import Vectors
from torchtext.data import TabularDataset
from torch.optim import adam
device = torch.device("cuda:0")
import nltk
nltk.download('punkt')

Load Training Data, Testing Data and Validation Data

In [ ]:
''' 
    /*----------------------------- LOAD_DATASET -------------
    | Function  : load_dataset()
    | Purpose   : Reads dataset(s) in CSV file format 
    | Arguments : 
    |       drive_path : Path to dataset file
    |       dataset    : Dataset file name
    | Return    :
    |       dataset    : Dataset in dataframe format
    *---------------------------------------------------------*/
'''
def load_dataset(dataset):
  loaded_dataset = pd.read_csv(dataset)     # Read CSV file
  print("="*40, "\n", loaded_dataset)                    # Print the dataset that we load in previous step

Understand and Pre-process Training Data, Testing Data and Validation Data

In [ ]:
''' 
    /*----------------------------- DATA_PRE-PROCESSING -------------
    | Function  : data_pre-processing()
    | Purpose   : Performs following pre-processing:
    |              •	Remove non-alphanumeric characters
    |              •	Lower case
    |              •	Remove leading and trailing whitespaces
    | Arguments : 
    |       text: Text to be pre-processed
    | Return    :
    |       text: Pre-processed text
    *------------------------------------------------------------------------------------------------*/
'''
def data_pre_processing(text):
      text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # Remove non alphanumeric character
      text = text.lower()                        # Lowercase all text
      return text.strip()                        # Remove leading and trailing whitespaces

Tokenize Text

In [ ]:
''' 
    /*----------------------------- TOKENIZE_TEXT -------------
    | Function  : data_tokenization()
    | Purpose   : Tokenizes a Text
    | Arguments : 
    |       text: Text to be tokenized
    | Return    :
    |       text: Tokenized Text
    *------------------------------------------------------------------------------------------------*/
'''
from nltk.tokenize import word_tokenize

def data_tokenization(s):
    return word_tokenize(s.lower())        # Apply pre-processing function (created in previous step) and tokenizer on text

Build Training Data, Testing Data and Validation Data Objects

In [ ]:
''' 
    /*----------------------------- BUILD_DATA_OBJECTS -------------
    | Function  : data_objects()
    | Purpose   : Build pre-processed and tokenized data objects
    | Arguments : 
    |       drive_path : Path of the directory where data files are placed
    | Return    :
    |       pre_processed_training_data, pre_processed_validation_data, pre_processed_testing_data, LABEL, TEXT
    *------------------------------------------------------------------------------------------------*/
'''
def data_objects():
  # Declared a Field object 
  # Field : A class that stores information about the way of preprocessing
  TEXT  = Field(sequential= True, tokenize = data_tokenization, lower = True, include_lengths = False, batch_first = False, init_token = '<sos>', 
            eos_token = '<eos>')
  LABEL = data.LabelField(dtype = torch.float)
  # TabularDataset : Defines a dataset of columns. Create a TabularDataset given a path, file format, 
  # and Field list
  training_data, validation_data, testing_data = TabularDataset.splits(path = '../Dataset/',
                                                            train       = 'Train_Data.csv', 
                                                            validation  = 'Validation_Data.csv',
                                                            test        = 'Test_Data.csv',
                                                            format      = 'csv', 
                                                            fields      = [('text', TEXT),('haveBugs', LABEL),('invalidPositionOverTime', LABEL),('implementationResponseIssue', LABEL),('invalidContextOverTime', LABEL),
                                                            ('interruptedEvent', LABEL),('invalidEventOccurraceOverTime', LABEL),('actionNotPossible', LABEL),('actionWhenNotAllowed', LABEL),('informationOutOfOrder', LABEL),
                                                            ('lackOfRequiredInformation', LABEL),('invalidInfoAccess', LABEL),('objectOutOfBoundForAnyState', LABEL),('objectOutOfBoundForSpecificState',LABEL),
                                                            ('artificialStupidity',LABEL), ('invalidValueChange',LABEL),('invalidGraphicalRespresentation',LABEL) ], 
                                                            skip_header = True)
  print("\nPre-processed and Tokenized Training Data:")
  print("\n=========================================")
  for i in range(len(training_data)):
    print(training_data[i].text)
  print("\nPre-processed and Tokenized Validation Data:")
  print("\n=========================================")
  for i in range(len(validation_data)):
    print(validation_data[i].text)
  print("\nPre-processed and Tokenized Testing Data:")
  print("\n=========================================")
  for i in range(len(testing_data)):
      print(testing_data[i].text)
  
  return training_data, validation_data, testing_data, LABEL, TEXT

Load Pre-Trained Word Embedding Vectors

In [ ]:
''' 
    /*----------------------------- LOAD_WORD_EMBEDDING_VECTORS -------------
    | Function  : load_word_embedding_vectors()
    | Purpose   : Load pre-trained word embedding vectors from memory
    | Arguments : 
    |       drive_path : Path to word embedding vectors file
    | Return    :
    |       vectors     : Loaded word embedding vectors
    *------------------------------------------------------------------------------------------------*/
'''
def load_word_embedding_vectors():
  # Load word embedding vectors from memory 
  # I have downloaded the Glove word embedding vectors 100d from internet and saved in my drive 
  # To use that, I simply give the path of that file and read file in my program using vocab.Vectors function
  vectors = vocab.Vectors('glove.6B.100d.txt', '../kaggle/')
  return vectors

Build Vocabulary

In [ ]:
''' 
    /*----------------------------- BUILD_VOCABULARY -------------
    | Function  : build_vocabulary()
    | Purpose   : Build vocabulary from input data
    | Arguments : 
    |       pre_processed_training_data   : Pre-processed training data
    |       pre_processed_validation_data : Pre-processed validation data
    |       pre_processed_testing_data    : Pre-processed testing data
    |       vectors                       : Word embedding vectors 
    |       LABEL                         : LABEL object (Pre-processing applied on output)
    |       TEXT                          : TEXT object (Pre-processing applied on input)
    | Return    :
    |       word_embeddings               : Word embedding vectors mapped on data
    |       vocabulary_size               : Size of vocabulary
    *------------------------------------------------------------------------------------------------*/
'''

def build_vocabulary(training_data, validation_data , testing_data, vectors, LABEL, TEXT):
  TEXT.build_vocab(training_data, validation_data , testing_data, vectors=vectors, unk_init=torch.Tensor.normal_)   # Build vocabulary from input text
  LABEL.build_vocab(training_data, validation_data , testing_data)                   # Build vocabulary from output / labels (Encode all labels)
  
  print("\n=========================================")
  print("Output/Label word to index dictionary: ", LABEL.vocab.stoi)
  print("\n=========================================")
  print("Input Text word to index dictionary:\n ", TEXT.vocab.stoi,"\n")
  
  word_embeddings = TEXT.vocab.vectors   # Load vectors
  vocabulary_size = len(TEXT.vocab)      # Size of vocabulary
  return word_embeddings, vocabulary_size

Represent Training Data, Testing Data and Validation Data in Machine Understandable Format

In [ ]:
batch_size=1

In [ ]:
''' 
    /*----------------------------- REPRESENT_DATA_IN_MACHINE_UNDERSTANDABLE_FORMAT -------------
    | Function  : data_iterators()
    | Purpose   : To build input data (Training, validation and testing data) iterators 
    |             (It will convert data into machine understandable format and make data objects which we can iterate over during model training and testing)
    | Arguments : 
    |       pre_processed_training_data   : Pre-processed training data
    |       pre_processed_validation_data : Pre-processed validation data
    |       pre_processed_testing_data    : Pre-processed testing data
    | Return    :
    |       training_iterator   : Training data iterator object
    |       validation_iterator : Validation data iterator object
    |       testing_iterator    : Testing data iterator object
    *------------------------------------------------------------------------------------------------*/
'''

def data_iterators(training_data, validation_data, testing_data):
  # Iterators handle numericalizing, batching, packaging. Basically, it does all the heavy lifting necessary 
  # to pass the data to a neural network
  # BucketIterator : Defines an iterator that batches examples of similar lengths together to minimizes the amount of padding needed
  # By using "splits" it applies processing steps on all datasets equally

  training_iterator, validation_iterator, testing_iterator = data.BucketIterator.splits((training_data, validation_data, testing_data), batch_size=batch_size, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)
  print("\nTraining Data Tensors Form\n")
  print("="*30, "\n")
  for batch in training_iterator:
    print(batch.text)
  print("\nValidation Data Tensors Form\n")
  print("="*30, "\n")
  for batch in validation_iterator:
    print(batch.text)
  print("\nTesting Data Tensors Form\n")
  print("="*30, "\n")
  for batch in testing_iterator:
    print(batch.text)
  return training_iterator, validation_iterator, testing_iterator

In [ ]:
print("+============================Data Preparation============================+\n\n")
print("---Step 2: Load Training Data, Testing Data and Validation Data---")
print("\nTraining data before pre_processing")
original_training_data = load_dataset("../Dataset/Train_Data.csv")

print("\n\nValidation data before pre_processing")
original_validation_data = load_dataset("../Dataset/Validation_Data.csv")

print("\n\nTesting data before pre_processing")
original_testing_data = load_dataset("../Dataset/Test_Data.csv")
print("\n---Step 3: Understand and Pre-process Training Data, Testing Data and Validation Data---")
print("\n---Step 4: Represent Training Data, Testing Data and Validation Data in Machine Understandable Format---")
preprocessed_training_data, preprocessed_validation_data, preprocessed_testing_data, LABEL, TEXT = data_objects()
# Load word embedding vectors from memory
vectors = load_word_embedding_vectors()
# Build vocabulary
word_embeddings, vocabulary_size = build_vocabulary(preprocessed_training_data, preprocessed_validation_data, preprocessed_testing_data, vectors, LABEL, TEXT)
# Create iterator objects
training_iterator, validation_iterator, testing_iterator = data_iterators(preprocessed_training_data, preprocessed_validation_data, preprocessed_testing_data)

Execute the Training Phase:
Model Architecture

In [ ]:
''' 
    /*----------------------------- MODEL_ARCHITECTURE -------------
    | Class     : RNN()
    | Purpose   : To build the architecture of model to be trained
    *---------------------------------------------------------
    | nn.Module : Base class for all neural network modules. Your models should also subclass this class.
    |
    | Arguments:
    |      output_dim    : For output layer number of nodes in output layer will be same as 
    |                      number of outputs required in your problem
    |	     hidden_dim    : Size of the hidden layer. Here size of hidden_state of the RNN
    | 		 input_dim     : Size of the vocabulary containing unique words. Total number of unique words in sample data 
    |		   embedding_dim : Size of each embedding vector. Here embeddding dimension of GloVe word embedding 
    |                      vectors is 100 so embedding_dim = 100
    |		   weights       : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table
    *------------------------------------------------------------------------------------
    | Function  : forward()
    | Purpose   : This function will automatically start foward propogation when model object is called
    | Arguments :
    |     text  : Input text of shape = (num_sequences, batch_size)	
	  | Return:
	  |     hidden_state : Final model state learned from input text
    ------------------------------------------------------------------------------
'''

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, word_embeddings):
        
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_layer = nn.Embedding(input_dim, embedding_dim)          # Embedding layer shape
        # Assign pre-trained weights and train during model training 
        # So the weight will be updated during backpropogation(if you don't want to train them during model training set requires_grad = False))
        self.embedding_layer.weight = nn.Parameter(word_embeddings, requires_grad=True)      
        self.rnn_layer       = nn.RNN(embedding_dim, hidden_dim, num_layers=1) # We can implement multiple layers of RNN simply by changing num_layers value 
        self.linear_layer    = nn.Linear(hidden_dim, output_dim)               # Shape of linear layer
        
    def forward(self, text):

        h_0 = self.init_hidden()   # Initialize first hidden state to all zeros
        
        # Here we will map all the indexes present in the input sequence to the corresponding 
		    # word vector using our trained word_embedddings.
	      # embedded input of shape = (num_sequences, batch_size, embedding_dimension)
        embedded_vectors = self.embedding_layer(text)  
        print (embedded_vectors)                  
        output_state, hidden_state = self.rnn_layer(embedded_vectors, h_0)  # Apply RNN layer and start learning sequence of words
        hidden_state = self.linear_layer(hidden_state.squeeze(0))      # Apply the linear layer on output
        return torch.sigmoid(hidden_state)
    def init_hidden(self):                                             # init_hidden for each beginning of sentence
        h_0 = torch.zeros(1, batch_size, self.hidden_dim)
        return h_0

Hyperparameters Settings

In [ ]:
'''
/*---------------- INITIALIZE_PARAMETERS ------------------
'''
input_dimension     = len(TEXT.vocab)
embedding_dimension = 100
hidden_dimension    = 10
output_dimension    = 16
number_of_epochs    = 10

Create Model Object

In [ ]:
"""
/* ----------------------- MODEL_OBJECT -----------------
| Create the object of model class and pass parameters required: RNN()
|           Arguments : 
|               input_dimension     : (integer) dimension of input layer(vocabulary size)
|               output_dimension    : (integer) number of output layer nodes 
|               hidden_dimension    : (integer) number of nodes/units in hidden layer
|               embedding_dimension : (integer) dimension of embedded vector
*-------------------------------------------------------*/
"""
model = RNN(input_dimension, embedding_dimension, hidden_dimension, output_dimension, word_embeddings)
model

Initialize Optimizer and Loss Function

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 1e-2)   # Initialize the optimizer
criterion = nn.BCELoss()                     # Intialize loss function

Evaluation Measure

In [ ]:
from sklearn.metrics import jaccard_score           # Import multi-label Accuracy (or Jaccard index)

''' 
    /*----------------------------- CALCULATE_MULTI-LABEL_ACCURACY -------------
    | Function  : calculate_jaccard()
    | Purpose   : Calculate Multi-label Accuracy Score
    | Arguments : 
    |       prediction : Predicted values
    |       label      : Actual values
    | Return    :
    |       jaccard   : Multi-label Accuracy score
    *---------------------------------------------------------*/
'''
def calculate_jaccard(prediction, label):              # Calculate multi-label Accuracy (or Jaccard index)

    rounded_preds = torch.round(prediction)       
    true = label.detach().cpu().numpy().reshape(-1)
    pred = rounded_preds.detach().cpu().numpy().reshape(-1)

    jaccard=torch.tensor(jaccard_score(true,pred))

    return jaccard

In [ ]:
from sklearn.metrics import f1_score                   # Import f1_score
'''
    /*----------------------------- CALCULATE_F1_SCORE -------------
    | Function  : calculate_f1()
    | Purpose   : Calculate F1 Score
    | Arguments : 
    |       prediction : Predicted values
    |       label      : Actual values
    | Return    :
    |       f1_score(true,pred)   : F1 score
    *---------------------------------------------------------*/
'''
def calculate_f1(prediction, label):                   # Calculate f1_score

    rounded_preds = torch.round(prediction)       
    true = label.detach().cpu().numpy().reshape(-1)
    pred = rounded_preds.detach().cpu().numpy().reshape(-1)
    return f1_score(true,pred,average='macro')

In [ ]:
from sklearn.metrics import hamming_loss               # Import hamming_loss
'''
    /*----------------------------- CALCULATE_HAMMING_LOSS_SCORE -------------
    | Function  : calculate_hl()
    | Purpose   : Calculate Hamming Loss Score
    | Arguments : 
    |       prediction : Predicted values
    |       label      : Actual values
    | Return    :
    |       hamming_loss(true,pred)   : Hamming Loss Score
    *---------------------------------------------------------*/
'''
def calculate_hl(prediction, label):                   # Calculate hamming_loss

    rounded_preds = torch.round(prediction)       
    true = label.detach().cpu().numpy().reshape(-1)
    pred = rounded_preds.detach().cpu().numpy().reshape(-1)
    return hamming_loss(true,pred)

In [ ]:
def calculate_accuracy(y_pred, y_true):
    accuracy =torch.mean(torch.div(torch.sum(torch.eq(y_true, torch.round(y_pred)), dim=-1),torch.tensor(y_true.shape[1],dtype=float)))
    return accuracy

Calculate Epoch Elapsed Time

In [ ]:
''' 
    /*----------------------------- EPOCH_TIME_CALCULATION -------------
    | Function  : epoch_time()
    | Purpose   : Calculate time elapsed in each epoch
    | Arguments : 
    |        start_time   : Time when an epoch's execution starts
    |        end_time     : Time when an epoch's execution end
    | Return    :
    |        elapsed_mins : Time consumed by one epoch in minutes
    |        elapsed_secs : Time consumed by one epoch in seconds
    *---------------------------------------------------------*/
'''
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time                   # Time elapsed by one epoch 
    elapsed_mins = int(elapsed_time / 60)                  # Convert time in minutes
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60)) # Convert time in seconds
    return elapsed_mins, elapsed_secs

In [ ]:
epoch_loss      = 0                                                 # Initialize epoch loss to 0
epoch_accuracy  = 0                                                 # Initialize epoch accuracy to 0

model.train()         
# Start model training mode

for batch in training_iterator:
    optimizer.zero_grad()                                           # Clear all optimized gradients
    predictions = model(batch.text).squeeze(1)                     # Make model predictions on training data
    y_true=torch.stack((batch.haveBugs,batch.invalidPositionOverTime,batch.implementationResponseIssue,batch.invalidContextOverTime,batch.interruptedEvent,
    batch.invalidEventOccurraceOverTime,batch.actionNotPossible,batch.actionWhenNotAllowed,batch.informationOutOfOrder,batch.lackOfRequiredInformation,batch.invalidInfoAccess,batch.objectOutOfBoundForAnyState,
    batch.objectOutOfBoundForSpecificState,batch.artificialStupidity,batch.invalidValueChange,batch.invalidGraphicalRespresentation
    ),dim=1)

    loss     = criterion(predictions, y_true)                 # Calculate loss for each batch in epoch
    accuracy = calculate_accuracy(predictions, y_true) 
    break

Train Model

In [ ]:
''' 
    /*----------------------------- TRAIN_MODEL -------------
    | Function  : train()
    | Purpose   : Train Model
    | Arguments : 
    |        model                 : Model object
    |        training_data_iterator: Training data iterator object
    |        optimizer             : Optimization algorithm
    |        criterion             : Loss funtion
    | Return    :
    |        epoch_loss            : Train data loss at each epoch
    |        epoch_accuracy        : Train data accuracy at each epoch
    *---------------------------------------------------------*/
'''
def train(model, iterator, optimizer, criterion):
    
    epoch_loss      = 0                                                 # Initialize epoch loss to 0
    epoch_accuracy  = 0                                                 # Initialize epoch accuracy to 0
    epoch_hamming=0
    epoch_f1=0
    model.train()                                                       # Start model training mode
    
    for batch in iterator:
        
        optimizer.zero_grad()                                           # Clear all optimized gradients
        predictions = model(batch.text).squeeze(1)                     # Make model predictions on training data
        y_true=torch.stack((batch.haveBugs,batch.invalidPositionOverTime,batch.implementationResponseIssue,batch.invalidContextOverTime,batch.interruptedEvent,
    batch.invalidEventOccurraceOverTime,batch.actionNotPossible,batch.actionWhenNotAllowed,batch.informationOutOfOrder,batch.lackOfRequiredInformation,batch.invalidInfoAccess,batch.objectOutOfBoundForAnyState,
    batch.objectOutOfBoundForSpecificState,batch.artificialStupidity,batch.invalidValueChange,batch.invalidGraphicalRespresentation,),dim=1)

        loss     = criterion(predictions, y_true)                 # Calculate loss for each batch in epoch
        accuracy = calculate_accuracy(predictions, y_true)        # Calculate accuracy for each batch in epoch
        hamming=calculate_hl(predictions, y_true)                 # Calculate hamming for each batch in epoch
        f1=calculate_f1(predictions, y_true)                      # Calculate f1 for each batch in epoch
        
        loss.backward()                                                  # Start backward propogation
        optimizer.step()                                                 # Optimization of parameters
        
        epoch_loss      += loss.item()                                   # Add loss for all batches in one epoch
        epoch_accuracy  += accuracy.item()    
        epoch_hamming+=hamming.item()
        epoch_f1 +=f1.item()
        # Add accuracy for all batches in one epoch
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator), epoch_hamming / len(iterator),epoch_f1 / len(iterator)   # Average loss, accuracy, hamming, and f1 for one epoch and return

In [ ]:
''' 
    /*----------------------------- SAVE_MODEL -------------
    | Function  : save_model()
    | Purpose   : Save a trained model on your hard disk
    | Arguments : 
    |        drive_path: Path to the directory where the trained model will be saved
    | Return    :
    |        Trained model will be saved on hard disk
    *---------------------------------------------------------*/

'''
def save_model():
  torch.save(model.state_dict(), '../Model/' + 'best-model.pt')

 Evaluate Model

In [ ]:
''' 
    /*----------------------------- Evaluate_MODEL -------------
    | Function  : evaluate()
    | Purpose   : Function to be used in Validation and Test Phase
    | Arguments : 
    |        model                : Model object
    |        data_iterator:  Data iterator object
    | Return    :
    |        epoch_loss           : Data loss at each epoch
    |        epoch_accuracy       : Data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def evaluate(model, iterator, criterion):
    epoch_f1       =0       # Initialize epoch f1 to 0
    epoch_loss     = 0      # Initialize epoch loss to 0
    epoch_accuracy = 0      # Initialize epoch accuracy to 0
    epoch_hamming  =0.      # Initialize epoch hamming to 0
    model.eval()            # Start model evaluation mode
    
    with torch.no_grad():
    
        for batch in iterator:

            y_true=torch.stack((batch.haveBugs,batch.invalidPositionOverTime,batch.implementationResponseIssue,batch.invalidContextOverTime,batch.interruptedEvent,
    batch.invalidEventOccurraceOverTime,batch.actionNotPossible,batch.actionWhenNotAllowed,batch.informationOutOfOrder,batch.lackOfRequiredInformation,batch.invalidInfoAccess,batch.objectOutOfBoundForAnyState,
    batch.objectOutOfBoundForSpecificState,batch.artificialStupidity,batch.invalidValueChange,batch.invalidGraphicalRespresentation),dim=1)

            predictions = model(batch.text).squeeze(1) 
 
                # Make model predictions on data
            loss     = criterion(predictions, y_true)                 # Calculate loss for each batch in epoch
            accuracy = calculate_accuracy(predictions, y_true)        # Calculate accuracy for each batch in epoch
            f1 = calculate_f1(predictions, y_true)                    # Calculate f1 for each batch in epoch
            hamming=calculate_hl(predictions, y_true)                 # Calculate hamming for each batch in epoch
            epoch_loss += loss.item()                                 # Add loss for all batches, in one epoch
            epoch_accuracy += accuracy.item()                         # Add accuracy for all batches in one epoch
            epoch_f1+=f1.item()                                       # Add f1 for all batches in one epoch
            epoch_hamming=hamming.item()                              # Add hamming for all batches in one epoch
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator), epoch_hamming / len(iterator),epoch_f1 / len(iterator)  # Average loss, accuracy, hamming, and f1 for one epoch and return

Execute the Validation Phase

In [ ]:
''' 
    /*----------------------------- VALIDATE_MODEL -------------
    | Function  : validation()
    | Purpose   : Evalaute the performance of a trained  model
    | Arguments : 
    |        model                   : Model object
    |        validation_data_iterator: Validation data iterator object
    |        criterion               : Loss function
    | Return    :
    |        epoch_loss           : Validation data loss at each epoch
    |        epoch_accuracy       : Validation data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def validation(model, validation_iterator, criterion):
      best_validation_loss = float('inf')                                                        # Declare best validation loss variable
      validation_loss, validation_accuracy,ham,f1 = evaluate(model, validation_iterator, criterion)     # Start model validation phase
      
      if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        save_model()                                   # Save model on epoch where the validation loss is lowest
      return validation_loss, validation_accuracy,ham,f1

Execute the Testing Phase

In [ ]:
"""
/*---------------------- LOAD_SAVED_MODEL ----------
|  Function  : load_model()
|  Purpose   : Method to load previously saved model
|  Arguments :
|       drive_path : Path of directory where model is saved
|  Return    :
|              Saved model will be loaded in memory
*---------------------------------------------------------*/
"""
def load_model():
  return model.load_state_dict(torch.load('../Model/'+ 'best-model.pt'))  # Load pre-trained model

 Test Model

In [ ]:
''' 
    /*----------------------------- TEST_MODEL -------------
    | Function  : test()
    | Purpose   : Evalaute the performance of a trained  model
    | Arguments : 
    |        model                : Model object
    |        testing_data_iterator: Test data iterator object
    |        criterion            : Loss function
    | Return    :
    |        epoch_loss           : Test data loss at each epoch
    |        epoch_accuracy       : Test data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def test(model, testing_iterator, criterion):
  load_model()
  testing_loss, testing_accuracy,ham,f1 = evaluate(model, testing_iterator, criterion)   # Start model testing
  return testing_loss, testing_accuracy,ham,f1

Take Input from User and Convert it into Feature Vector Same as Training Data

Execute the Application Phase

In [ ]:
''' 
    /*----------------------------- USER_INPUT -------------
    | Function  : take_user_input()
    | Purpose   : Take unseen input from user
    | Arguments : 
    |        TEXT : Field object to apply pre-processing on input text (same as sample data)
    | Return    :
    |        user_comment_tensor : User input in machine understandable format
    |----------------------------------------------------------
    | - Let us now predict the emotion on a single SMS for the real time evaluation purpose 
    | 1 : Take input from user
    | 2 : Preprocess the user input
    | 3 : Fit vocabulary previously made for sample data on user input. The indexes assigned for words in 
    |     sample data will be assigned to user input. Words in user input that does not appear in 
    |     sample data will have zero value
    | 4 : Convert user input to an array
    | 5 : Make tensor from array. As pytorch only work with tensors
    *---------------------------------------------------------*/

'''

def take_user_input(TEXT):
  user_comment = input("Enter comment: ") 
  
  #Preprocess user input
  preprocessed_user_comment = TEXT.preprocess(user_comment)
  preprocessed_user_comment = [TEXT.init_token] + preprocessed_user_comment + [TEXT.eos_token]
  user_comment_vocabulary = [TEXT.vocab.stoi[x] for x in preprocessed_user_comment]
  user_comment_array = np.asarray(user_comment_vocabulary)
  user_comment_tensor = torch.LongTensor(user_comment_array).unsqueeze(1)
  user_comment_tensor = user_comment_tensor

  print("\nPreprocessed User_input:\n==========================")
  print(preprocessed_user_comment)
  print("\nIdx stored in vocab, corresponding to each word in user_input:\n==========================")
  print(user_comment_vocabulary) 
  print("\nUser_input as a tensor:\n==========================" )
  print(user_comment_tensor)
  
  return user_comment_tensor

Model Prediction

In [ ]:
"""
/*----------------------- MODEL_PREDICTION --------
|  Function  : model_prediction()
|  Purpose   : Use trained model to predict the output of unseen instances
|  Arguments : 
|       user_input : Input taken from user
|       drive_path : Path of the directory where trained model is saved
|  Return    : 
|       out     : Set of Emotion labels that best represent the mental state of a writer 
*--------------------------------------------------
|   1. Set the model to evaluation mode
|   2. Set all the gradients to zero
|   3. Apply trained model on user input
|   
|   4. torch.round() : 
|         Return the value rounded to the closest integer (0 or 1)
*-------------------------------------------------*/   
"""

def model_predictions(user_input):
  # Evaluate model
  load_model()  # Load model from memory to test its performance
  model.eval()
  with torch.no_grad():
    #print(Evaluate_text_tensor)
    # Model Prediction
    out = model(user_input)

  return out

In [ ]:
print("\n+=====================Execute the Training and Validation Phase=====================+\n\n")
# Step 5: Execute the Training Phase
import time
for epoch in range(number_of_epochs):

    start_time = time.time()                                    # Start time when one epoch will start executing
    
    training_loss, training_accuracy,train_ham,train_f1 = train(model, training_iterator, optimizer, criterion)   # Start model training
    
    # Step 7: Execute the Validation Phase
    validation_loss, validation_accuracy,val_ham,val_f1 = validation(model, validation_iterator, criterion)
    
    end_time = time.time()                                       # End time when one epoch will end executing
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)    # Calculate time consumed by one epoch (in minutes and seconds)
      
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTraining Loss: {training_loss:.3f}   | Training Accuracy: {training_accuracy*100:.2f} | Training Hamming: {train_ham*100:.2f}| Training F1: {train_f1*100:.2f}%')
    print(f'\tValidation Loss: {validation_loss:.3f} |  Validation Accuracy: {validation_accuracy*100:.2f}| Val Hamming: {val_ham*100:.2f}| Val F1: {val_f1*100:.2f}%')



In [ ]:
print("\n+=====================Execute the Testing Phase=====================+\n\n")
# Step 8: Execute the Testing Phase

testing_loss, testing_accuracy,test_ham,test_f1 = test(model, testing_iterator, criterion)
print(f'Testing Loss: {testing_loss:.3f} | Testing Accuracy: {testing_accuracy*100:.2f}| test Hamming: {val_ham*100:.2f}| test F1: {val_f1*100:.2f}%')


Take input

In [ ]:
print("\n+===================Execute the Application Phase===================+\n\n")

# Step 7: Execute the Application Phase

user_input = take_user_input(TEXT)   # Take unseen input from user
Label = model_predictions(user_input)  # Make trained model predictions on user input


label_list=['haveBugs','invalidPositionOverTime','implementationResponseIssue','invalidContextOverTime','interruptedEvent','invalidEventOccurraceOverTime','actionNotPossible','actionWhenNotAllowed','informationOutOfOrder','lackOfRequiredInformation','invalidInfoAccess','objectOutOfBoundForAnyState','objectOutOfBoundForSpecificState','artificialStupidity','invalidValueChange','invalidGraphicalRespresentation']
print("\n+===================Trained Model Prediction===================+\n\n")
# print(label_list[np.argmax(Label.cpu().numpy(),axis=1)[0]])
print([i for i,j in zip(label_list,list(Label.cpu().numpy()[0])) if j>0.50]) # Threshold for the probability (scoring into a class label)